# TASK5 - models

Propose other prediction models than the one proposed by us.

##### Roche team used Random Forest Classifier from Ensemble Methods

In a nutshell Random Forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting

----------------------------------------------------------------------------------------------------------------------

### LOAD PACKAGES

In [152]:
import sys  
sys.path.insert(0, '../src/')

In [208]:
import random
import time
import datacompy
import sys
import IPython
from IPython import display
import matplotlib 
import pandas as pd
import numpy as np 
import scipy as sp 
import sklearn
import pickle
from collections import Counter
import pickle as pkl
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
mpl.style.use('ggplot')
sns.set_style('dark')
pylab.rcParams['figure.figsize'] = 14,5
seed =40
plt.style.use('fivethirtyeight')

In [154]:
import functions
from functions import missing_ratio

In [155]:
all_df = pd.read_csv("../data/all_df.csv")

In [156]:
all_df.shape

(1299, 18)

----------------------------------------------------------------------------------------------------------------------

#### Model that i have choosen is based on my own knowladge and many publications of which one is more worth mentioning than others: https://www.cs.cornell.edu/~caruana/ctp/ct.papers/caruana.icml06.pdf 

### Split test and train data

I will use sklearn function to split the training data in two datasets and it will be classic 75/25 split.

In [157]:
seed = 40

In [158]:
train = all_df.iloc[:881, :]

In [159]:
train.tail()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f
876,887,0.0,2,808,1,27.0,0,0,13.00,2,1,1,0,1,2,0,0,0
877,888,1.0,1,461,0,19.0,0,0,30.00,2,1,1,0,2,3,0,0,0
878,889,0.0,3,604,0,28.0,1,2,23.45,2,4,0,0,2,3,0,1,0
879,890,1.0,1,113,1,26.0,0,0,30.00,0,1,1,0,3,3,0,0,0
880,891,0.0,3,338,1,32.0,0,0,7.75,1,1,1,0,3,1,0,0,0


In [160]:
def nans(df): return df[df.isnull().any(axis=1)]

In [161]:
nans(train)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f


In [162]:
missing_ratio(train)

,"Missing value, %",N unique value,dtype
passengerid,0.0,881,int64
survived,0.0,2,float64
pclass,0.0,3,int64
name,0.0,881,int64
sex,0.0,2,int64
age,0.0,88,float64
sibsp,0.0,6,int64
parch,0.0,7,int64
fare,0.0,247,float64
embarked,0.0,3,int64


In [163]:
test  = all_df.iloc[881:, :]

In [164]:
test.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f
881,892,NaN,3,635,1,34.5,0,0,7.8292,1,1,1,0,3,1,0,0,0
882,893,NaN,3,1256,0,47.0,1,0,7.0000,2,2,0,0,4,1,1,0,0
883,894,NaN,2,834,1,62.0,0,0,9.6875,1,1,1,0,3,2,0,0,0
884,895,NaN,3,1268,1,27.0,0,0,8.6625,2,1,1,0,3,2,0,0,0
885,896,NaN,3,540,0,22.0,1,1,12.2875,2,3,0,0,4,2,0,1,0


In [165]:
missing_ratio(test)

,"Missing value, %",N unique value,dtype
passengerid,0.0,418,int64
survived,100.0,0,float64
pclass,0.0,3,int64
name,0.0,418,int64
sex,0.0,2,int64
age,0.0,79,float64
sibsp,0.0,7,int64
parch,0.0,8,int64
fare,0.0,169,float64
embarked,0.0,3,int64


In [166]:
x_full = pd.concat([train.drop('survived', axis = 1), test.drop('survived', axis = 1)], axis = 0)

In [167]:
x_full.shape

(1299, 17)

In [168]:
x_full.drop('passengerid', axis = 1, inplace=True)

In [169]:
x_full.isnull().sum()

pclass           0
name             0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         0
familysize       0
isalone          0
no_fare          0
title            0
fare_category    0
small_f          0
med_f            0
large_f          0
dtype: int64

In [170]:
x_full.shape

(1299, 16)

In [171]:
x_dummies = pd.get_dummies(x_full, drop_first= True)

In [172]:
x_dummies.shape

(1299, 16)

In [173]:
x_dummies.dtypes

pclass             int64
name               int64
sex                int64
age              float64
sibsp              int64
parch              int64
fare             float64
embarked           int64
familysize         int64
isalone            int64
no_fare            int64
title              int64
fare_category      int64
small_f            int64
med_f              int64
large_f            int64
dtype: object

In [174]:
X = x_dummies[:len(train)]; new_x = x_dummies[len(train):]

In [175]:
y = train.survived

In [176]:
from sklearn.model_selection import train_test_split

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = .3,
                                                    random_state = 5,
                                                   stratify = y)

In [178]:
from xgboost import XGBClassifier

In [179]:
xgb = XGBClassifier()

In [180]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [181]:
xgb.score(X_test, y_test)

0.8075471698113208

In [182]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

In [183]:
# the parameter grid 
gbm_param_grid = {
    'n_estimators': range(8, 20),
    'max_depth': range(6, 10),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}

In [184]:
# instantiate the regressor
gbm = XGBClassifier(n_estimators=10)

In [185]:
# perform random search
xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid, 
                                    estimator = gbm, scoring = "accuracy", 
                                    verbose = 1, n_iter = 50, cv = 4)

In [186]:
# fit randomized_mse
xgb_random.fit(X, y)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.9s finished


RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_co...
                                           subsample=None, tree_method=None,
                                           validate_parameters=False,
                                           verbosity=None),
                 

In [214]:
filename = 'model_klimarczyk.pkl'
model_pickle = open(f"../data/{filename}", 'wb')
pickle.dump(gbm, model_pickle)
model_pickle.close()

In [215]:
# best parameters and lowest RMSE
print("Best parameters found: ", xgb_random.best_params_)
print("Best accuracy found: ", xgb_random.best_score_)

Best parameters found:  {'n_estimators': 14, 'max_depth': 6, 'learning_rate': 0.55, 'colsample_bytree': 0.8}
Best accuracy found:  0.8388195232690124


In [188]:
xgb_pred = xgb_random.predict(new_x)

In [222]:
# load the model
loaded_model = pickle.load(open(f"../data/{filename}", 'rb'))
loaded_model.fit(X_train, y_train)
loaded_model.score(X_test, y_test)

0.8452830188679246

-------------

In [189]:
tpid = test.passengerid

In [190]:
type(tpid)

pandas.core.series.Series

In [191]:
tpid = tpid.to_frame() 

In [193]:
tpid.columns = ['PassengerId']

In [194]:
type(tpid)

pandas.core.frame.DataFrame

In [195]:
pdfxgb = pd.DataFrame(xgb_pred)

In [196]:
pdfxgb.columns = ["Survived"]

In [197]:
pdfxgb.shape

(418, 1)

In [198]:
type(pdfxgb)

pandas.core.frame.DataFrame

In [199]:
submission = pd.concat([tpid.reset_index(drop=True), pdfxgb.reset_index(drop=True)], axis = 1).reset_index(drop=True)

In [200]:
submission

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [201]:
submission.to_csv('../data/titanic_submission.csv', header = True, index = False)

In [202]:
test_org = pd.read_csv('../data/test.csv')

In [203]:
validation = pd.merge(test_org.reset_index(drop=True), submission.reset_index(drop=True), on='PassengerId')
validation

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1.0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0.0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0.0


In [204]:
validation.to_csv('../data/klimarczyk_validation.csv', header = True, index = False)